In [71]:
#import albumentations
import cv2
import numpy as np 
import os 
import pandas as pd 
from PIL import Image
import re

import torch 
import torchvision

In [19]:
# Model imports 

from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator

from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SequentialSampler

In [8]:
DATA_PATH = 'data/smartwarehouseyolo/SmartWarehouseYOLO/'

In [13]:
len(os.listdir(os.path.join(DATA_PATH, 'data')))

1998

In [28]:
with open(os.path.join(DATA_PATH, 'smartwarehouse_train.txt'), 'r') as trainfile:
    file = trainfile.read()

In [76]:
file = open(os.path.join(DATA_PATH, 'data/000002.txt'), 'r')
_res = list()
for line in file:
    line  = re.sub('\n', ' ', line)
    line[2:-1].split(' ')
    _res.append(line[2:-1].split(' '))

In [80]:
def get_boxes(txt_file):
    file = open(txt_file, 'r')
    _res = list()
    for line in file:
        line = re.sub('\n', ' ', line)
        _res.append(line[2:-1].split(' ') )
    return _res        

In [83]:
class SmartWareHouseDataset(Dataset):
    
    def __init__(self,  image_dir, transforms=None):
        super().__init__()
        self.txt_file = txt_file
        self.image_dir = image_dir 
        self.transforms = transforms
        self.image_ids = [ f'{el:06}' for el in range(1000)]
    
    def __getitem__(self, index: int):
        image_id = self.image_ids[index]
        image =  cv2.imread(os.path.join(self.image_dir, f'{image_id}.jpg'))
        # Convert image 
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image /= 255.0
        
        
        box = get_boxes(os.path.join(self.image_dir, f'{image_id}.txt'))
        n_boxes = len(box)
        assert n_boxes > 0, 'There are no boxes on provided image'
        
        if self.transforms:
            sample = {
                'image' : image,
            }
            sample = self.transforms(**sample)
            image = sample['image']
        return image, image_id, box 
    
    
    